In [1]:
from ldaclass import LDApreprocess

import sys, os, nltk, re, argparse, spacy, pyLDAvis, gensim, warnings, time, pickle
import json, re
import numpy as np
import pandas as pd
import gensim.corpora as corpora
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from gensim.models import LdaMulticore
from gensim.corpora.dictionary import Dictionary
import pyLDAvis.gensim
from tqdm import tqdm
import matplotlib.pyplot as plt
from argparse import ArgumentParser
from konlpy.tag import Mecab
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer

from collections import Counter
import itertools
import copy
from gensim.models import TfidfModel
import random
import pprint
from sklearn.feature_extraction.text import CountVectorizer


from gensim.models import FastText

/home/literai-deploy/anaconda3/envs/gale/lib/python3.7/site-packages/scipy/sparse/sparsetools.py:21: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()


In [2]:
mecab = Mecab()

In [34]:
def kor_preprocess(context):
    words = []
    def tokenize(text):
        for x in tqdm(text):
            try:
                words.append(mecab.nouns(x))
            except Exception as e:
                print(x, e)
        return words

    def remove_stopwords(words):
        return [[word for word in simple_preprocess(str(word_list))] for word_list in tqdm(words)]

    print("tokenizing...")
    data_words = tokenize(context)
    print("remove stopwords...")
    tokenized_context = remove_stopwords(data_words)
    return tokenized_context



def make_ngram(texts, n):
    results = []
    bigram = gensim.models.Phrases(texts, min_count=5, threshold=10)

    if n == 2:
        for doc in tqdm(texts):
            results.append(bigram[doc])
    elif n==3:
        trigram = gensim.models.Phrases(bigram[texts])
        for doc in tqdm(texts):
            results.append(trigram[doc])
    return results

def tune_params(_corpus, _id2word, max_topics, min_topics = 1):
    coherence = []
    perplexity = []
    models = [ ]

    for i in range(min_topics, max_topics+1):
        print("n_topics = ",i)
        _lda_model = LdaMulticore(corpus=_corpus, num_topics=i,
                                 id2word=_id2word, workers=6, iterations=400, passes=2)
        pprint.pprint(_lda_model.print_topics())
        print("get coherence, perplexity...")
        cm = CoherenceModel(model=_lda_model, corpus=_corpus, coherence='u_mass')
        coh = cm.get_coherence()
        per = _lda_model.log_perplexity(_corpus)

        coherence.append(coh)
        perplexity.append(per)
        models.append(_lda_model)

        print("coherence  =  ", coh)
        print("perplexity = ", per)

        print("-"*40)



    plt.plot(coherence, c='r')
    plt.plot(perplexity)
    plt.show()

    return coherence, perplexity, models

def remove_oversamples(texts, n, top_n):
    #top_n => 상위 몇 개의 단어를 줄일 것인지
    #n => 몇 개로 줄일 것인지
    import copy
    copy = copy
    _texts = copy.deepcopy(texts)

    flatten_texts = list(itertools.chain.from_iterable(_texts))
    texts_dict = dict(Counter(flatten_texts))
    res = sorted(texts_dict.items(), key=(lambda x:x[1]), reverse=True)

    for i in tqdm(range(top_n)):
        print("\n", i+1,"번째", res[i])


        indexes = [index for index, value in enumerate(_texts) if res[i][0] in value]

        count = 0
        for idx in indexes:
            count += _texts[idx].count(res[i][0])
            if(count > n):
                break

        print(len(indexes), idx)
        chages = [[token for token in _texts[index] if token != res[i][0]] for index in indexes if index >= idx]
        index = [index for index in indexes if index >= idx]


        count = 0
        for x, y in zip(index, chages):
            _texts[x] = y
            count += 1

        print(len(indexes),"개 중", count,"개 수정 되었습니다.")

    return _texts
        

def remove_oversamples_with_dict(texts, vocab_dict, top_n, max_count = None):
    result = list()
    if max_count:
        vocab = [k for k, c in vocab_dict if c <= max_count]
    else:
        vocab = [k for k, c in vocab_dict[top_n:]]

    for text in tqdm(texts):
        temp_text = [word for word in text if word in vocab]
        if len(temp_text):
            result.append(temp_text)
    return result

def remove_undersaples_with_dict(texts, vocab_dict, bottom_n, min_count = None):
    result = list()
    if min_count:
        vocab = [k for k, c in vocab_dict if c >= min_count]
    else:
        vocab = [k for k, c in vocab_dict[:-bottom_n]]
    for text in tqdm(texts):
        temp_text = [word for word in text if word in vocab]
        if len(temp_text):
            result.append(temp_text)
    return result

def get_top_n(_texts):
    flatten_texts = list(itertools.chain.from_iterable(texts))
    texts_dict = dict(Counter(flatten_texts))
    res = sorted(texts_dict.items(), key=(lambda x:x[1]), reverse=True)
    return res

def get_bottom_n(_texts):
    flatten_texts = list(itertools.chain.from_iterable(texts))
    texts_dict = dict(Counter(flatten_texts))
    res = sorted(texts_dict.items(), key=(lambda x:x[1]), reverse=False)
    return res

def get_tfidf_corpus(_corpus):
    tfidf = TfidfModel(corpus)
    corpus_tfidf = tfidf[corpus]
    return corpus_tfidf

def get_id2word(texts):
    return corpora.dictionary.Dictionary(texts)

def get_corpus(texts, id2word):
    return [id2word.doc2bow(text) for text in tqdm(texts)]





In [4]:
with open('음료.pkl', 'rb') as f:
    reviews = pickle.load(f)

In [5]:
context = reviews#.sample(10000)
context = context['리뷰'].values

In [7]:
texts = kor_preprocess(context)

  0%|          | 5909/2897557 [00:00<01:38, 29502.45it/s]

tokenizing...


 53%|█████▎    | 1548627/2897557 [01:22<01:11, 18819.70it/s]

항상시켜먹어용 (∩R467$#R467$∩) not enough values to unpack (expected 2, got 1)


 58%|█████▊    | 1668118/2897557 [01:27<00:47, 25869.67it/s]

...

 69%|██████▉   | 2005046/2897557 [01:43<00:33, 26808.51it/s]

머그잔 예쁘네요. 근데 두개까지는 필요없구..  머그 한개 넣어주고 가격을 낮추면 좋겠네요 �5H�5H not enough values to unpack (expected 2, got 1)


  1%|          | 31260/2897554 [00:00<01:42, 27998.61it/s] 

remove stopwords...


100%|██████████| 2897554/2897554 [00:17<00:00, 163397.75it/s]


In [8]:
with open('preprocessed_음료', 'wb') as f:
    pickle.dump(texts, f)

In [35]:
with open('preprocessed_음료', 'rb') as f:
    texts = pickle.load(f)

In [36]:
count_dict = get_top_n(texts)

In [37]:
len(count_dict)

39493

In [38]:
get_top_n(texts)

[('배송', 834197),
 ('구매', 492670),
 ('가격', 299221),
 ('감사', 262453),
 ('만족', 257690),
 ('주문', 209591),
 ('커피', 153882),
 ('포장', 144575),
 ('제품', 127671),
 ('구입', 124868),
 ('상품', 96747),
 ('조아', 73773),
 ('사용', 65050),
 ('다음', 59529),
 ('여기', 59196),
 ('추천', 57498),
 ('상태', 54180),
 ('박스', 53469),
 ('기간', 50690),
 ('생각', 48483),
 ('최고', 46666),
 ('작성', 46653),
 ('이용', 45567),
 ('유통', 41308),
 ('안드로이드', 40631),
 ('생수', 39323),
 ('마트', 39028),
 ('기한', 38323),
 ('도움', 35120),
 ('콜라', 34542),
 ('후기', 34424),
 ('선물', 34112),
 ('음료', 33489),
 ('하나', 31334),
 ('아이', 31007),
 ('대비', 30634),
 ('이번', 30412),
 ('택배', 29986),
 ('도착', 29373),
 ('탄산', 27971),
 ('물맛', 27663),
 ('탄산수', 27419),
 ('번창', 26845),
 ('마음', 26633),
 ('하루', 26335),
 ('사무실', 26185),
 ('애용', 26111),
 ('크기', 25655),
 ('마켓', 25556),
 ('가성', 24706),
 ('물건', 24289),
 ('맥심', 23859),
 ('여름', 23829),
 ('유효', 23617),
 ('사은품', 23432),
 ('효과', 23065),
 ('의향', 22642),
 ('건강', 22451),
 ('처음', 21584),
 ('배달', 21548),
 ('정도', 21401),
 ('옥션', 

In [39]:
texts=remove_oversamples(texts, n=20000, top_n=25)#25

  0%|          | 0/25 [00:00<?, ?it/s]


 1 번째 ('배송', 834197)
732908 63164


  4%|▍         | 1/25 [00:01<00:25,  1.05s/it]

732908 개 중 714933 개 수정 되었습니다.

 2 번째 ('구매', 492670)
384976 166580


  8%|▊         | 2/25 [00:03<00:35,  1.54s/it]

...

42293 1189154
42293 개 중 23728 개 수정 되었습니다.

 24 번째 ('유통', 41308)


 96%|█████████▌| 24/25 [00:17<00:00,  1.72it/s]

39297 1439596
39297 개 중 20297 개 수정 되었습니다.

 25 번째 ('안드로이드', 40631)


100%|██████████| 25/25 [00:18<00:00,  1.39it/s]

40611 1215403
40611 개 중 20612 개 수정 되었습니다.


In [40]:
get_top_n(texts)

[('생수', 39323),
 ('마트', 39028),
 ('기한', 38323),
 ('도움', 35120),
 ('콜라', 34542),
 ('후기', 34424),
 ('선물', 34112),
 ('음료', 33489),
 ('하나', 31334),
 ('아이', 31007),
 ('대비', 30634),
 ('이번', 30412),
 ('택배', 29986),
 ('도착', 29373),
 ('탄산', 27971),
 ('물맛', 27663),
 ('탄산수', 27419),
 ('번창', 26845),
 ('마음', 26633),
 ('하루', 26335),
 ('사무실', 26185),
 ('애용', 26111),
 ('크기', 25655),
 ('마켓', 25556),
 ('가성', 24706),
 ('물건', 24289),
 ('맥심', 23859),
 ('여름', 23829),
 ('유효', 23617),
 ('사은품', 23432),
 ('효과', 23065),
 ('의향', 22642),
 ('건강', 22451),
 ('처음', 21584),
 ('배달', 21548),
 ('정도', 21401),
 ('옥션', 21349),
 ('횟수', 21314),
 ('카누', 20916),
 ('회사', 20432),
 ('단골', 20375),
 ('여기', 20000),
 ('다음', 20000),
 ('배송', 20000),
 ('제품', 20000),
 ('포장', 20000),
 ('유통', 20000),
 ('감사', 20000),
 ('주문', 20000),
 ('상품', 20000),
 ('구입', 20000),
 ('최고', 20000),
 ('가격', 20000),
 ('만족', 20000),
 ('생각', 20000),
 ('상태', 20000),
 ('추천', 20000),
 ('박스', 20000),
 ('작성', 20000),
 ('안드로이드', 20000),
 ('사용', 19999),
 ('기간', 19999),
 (

In [18]:
texts = remove_undersaples_with_dict(texts=texts, vocab_dict=count_dict, bottom_n=5, min_count=10)

100%|██████████| 1807640/1807640 [01:02<00:00, 28752.26it/s]


In [16]:
get_bottom_n(texts)

[('도민', 5),
 ('미끈', 5),
 ('다디', 5),
 ('문량', 5),
 ('월자', 5),
 ('동문서답', 5),
 ('물문', 5),
 ('얼음덩어리', 5),
 ('청계', 5),
 ('운반비', 5),
 ('교오', 5),
 ('조한', 5),
 ('하항', 5),
 ('쌍욕', 5),
 ('지반', 5),
 ('나삼', 5),
 ('상도', 5),
 ('서림', 5),
 ('수서', 5),
 ('원효대사', 5),
 ('자라목', 5),
 ('코웨이', 5),
 ('응수', 5),
 ('부동', 5),
 ('실비', 5),
 ('국산물', 5),
 ('반칙', 5),
 ('토지', 5),
 ('마그마', 5),
 ('앞배', 5),
 ('무리수', 5),
 ('트위스트', 5),
 ('역수', 5),
 ('루시', 5),
 ('홍어', 5),
 ('하룬가', 5),
 ('브랜디', 5),
 ('만을', 5),
 ('스퍼', 5),
 ('애물', 5),
 ('게옴', 5),
 ('밥그릇', 5),
 ('역식', 5),
 ('뇌혈관', 5),
 ('만용', 5),
 ('장창', 5),
 ('래핑', 5),
 ('응디', 5),
 ('생명력', 5),
 ('우호', 5),
 ('수문', 5),
 ('오케', 5),
 ('부상', 5),
 ('미까', 5),
 ('어업', 5),
 ('명절날', 5),
 ('박산', 5),
 ('맴맴', 5),
 ('이하일', 5),
 ('시추', 5),
 ('야서', 5),
 ('종가', 5),
 ('동탄', 5),
 ('두껍', 5),
 ('아제', 5),
 ('뇌리', 5),
 ('초음', 5),
 ('사단', 5),
 ('개편', 5),
 ('뵈송', 5),
 ('여행사', 5),
 ('사라라', 5),
 ('보물찾기', 5),
 ('행가', 5),
 ('마추', 5),
 ('당월', 5),
 ('월아', 5),
 ('발산', 5),
 ('스탠드', 5),
 ('파이프', 5),
 ('항만', 5),


In [41]:
id2word = get_id2word(texts=texts)
corpus = get_corpus(texts=texts, id2word=id2word)

100%|██████████| 2897554/2897554 [00:09<00:00, 293134.19it/s]


In [42]:
lda_model = LdaMulticore(corpus=corpus, num_topics=9, id2word=id2word, workers=6, iterations=400, passes=10)
lda_model.print_topics()

[(0,
  '0.101*"선물" + 0.062*"이번" + 0.057*"사은품" + 0.049*"하루" + 0.027*"성비" + 0.026*"유효" + 0.025*"건데" + 0.024*"마켓" + 0.022*"횟수" + 0.021*"옥션"'),
 (1,
  '0.079*"물맛" + 0.028*"믹스" + 0.028*"기대" + 0.025*"쿠폰" + 0.025*"크기" + 0.024*"텀블러" + 0.023*"다이어트" + 0.021*"회사" + 0.020*"콜라" + 0.020*"대신"'),
 (2,
  '0.058*"기한" + 0.039*"카누" + 0.032*"맥심" + 0.027*"라떼" + 0.020*"원두" + 0.018*"건강" + 0.018*"강추" + 0.018*"디카페인" + 0.017*"모카" + 0.016*"골드"'),
 (3,
  '0.056*"애용" + 0.036*"기분" + 0.035*"음료" + 0.032*"물건" + 0.032*"의사" + 0.031*"만시" + 0.028*"여름" + 0.027*"레몬" + 0.026*"탄산수" + 0.025*"사이즈"'),
 (4,
  '0.110*"생수" + 0.052*"택배" + 0.046*"배달" + 0.040*"기사" + 0.030*"카페" + 0.026*"친절" + 0.016*"시간" + 0.015*"샘물" + 0.012*"고급" + 0.012*"문제"'),
 (5,
  '0.081*"대비" + 0.069*"사무실" + 0.067*"마음" + 0.064*"번창" + 0.047*"품질" + 0.043*"우유" + 0.040*"수고" + 0.025*"노브랜드" + 0.023*"우리" + 0.023*"가지"'),
 (6,
  '0.055*"효과" + 0.053*"처음" + 0.040*"이거" + 0.039*"아침" + 0.033*"엄마" + 0.026*"겨울" + 0.023*"편리" + 0.022*"비염" + 0.019*"빠름" + 0.019*"주스"'),
 (7,
  '0.071*"가

In [55]:
lda_model = LdaMulticore(corpus=corpus, num_topics=9, id2word=id2word, workers=6, iterations=400, passes=9)
lda_model.print_topics()

[(0,
  '0.038*"가성" + 0.032*"카누" + 0.027*"맥심" + 0.025*"번창" + 0.022*"라떼" + 0.019*"믹스" + 0.018*"하나" + 0.017*"느낌" + 0.017*"이것" + 0.016*"다이어트"'),
 (1,
  '0.073*"애용" + 0.066*"하루" + 0.046*"이거" + 0.044*"아침" + 0.037*"여름" + 0.032*"한잔" + 0.029*"겨울" + 0.028*"용량" + 0.026*"제조" + 0.023*"부모"'),
 (2,
  '0.143*"물맛" + 0.092*"대비" + 0.065*"음료" + 0.053*"기대" + 0.050*"카페" + 0.047*"탄산" + 0.040*"탄산수" + 0.030*"보이차" + 0.027*"향도" + 0.024*"산수"'),
 (3,
  '0.073*"이번" + 0.065*"택배" + 0.049*"기사" + 0.049*"도움" + 0.039*"필요" + 0.034*"회사" + 0.032*"친절" + 0.031*"의향" + 0.030*"건데" + 0.029*"후기"'),
 (4,
  '0.112*"생수" + 0.034*"티백" + 0.029*"때문" + 0.022*"보리차" + 0.021*"비염" + 0.018*"삼다수" + 0.018*"노브랜드" + 0.017*"우리" + 0.016*"샘물" + 0.015*"리터"'),
 (5,
  '0.078*"이상" + 0.063*"사이즈" + 0.044*"게요" + 0.042*"주스" + 0.042*"빠름" + 0.041*"완전" + 0.037*"무료" + 0.035*"서비스" + 0.031*"총알" + 0.028*"냉장고"'),
 (6,
  '0.071*"아이" + 0.054*"효과" + 0.054*"배달" + 0.045*"물건" + 0.036*"크기" + 0.035*"쿠폰" + 0.035*"우유" + 0.031*"유효" + 0.029*"단골" + 0.028*"기분"'),
 (7,
  '0.074*"선